In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix
import pickle
from difflib import get_close_matches

# Load dataset
dataset = pd.read_csv('/content/drive/MyDrive/medicine recommendation system/Training.csv')

# Train-test split
X = dataset.drop('prognosis', axis=1)
y = dataset['prognosis']

# Encoding prognosis
le = LabelEncoder()
le.fit(y)
Y = le.transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=20)

# Train models
models = {
    'SVC': SVC(kernel='linear'),
    'RandomForest': RandomForestClassifier(n_estimators=100, random_state=42),
    'GradientBoosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
    'KNeighbors': KNeighborsClassifier(n_neighbors=5),
    'MultinomialNB': MultinomialNB()
}

for model_name, model in models.items():
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    print(f"{model_name} Accuracy: {accuracy}")

# Select SVC as the primary model
svc = SVC(kernel='linear')
svc.fit(X_train, y_train)

# Save model
pickle.dump(svc, open('svc.pkl', 'wb'))

# Load supporting data
sym_des = pd.read_csv("/content/drive/MyDrive/medicine recommendation system/symtoms_df.csv")
precautions = pd.read_csv("/content/drive/MyDrive/medicine recommendation system/precautions_df.csv")
workout = pd.read_csv("/content/drive/MyDrive/medicine recommendation system/workout_df.csv")
description = pd.read_csv("/content/drive/MyDrive/medicine recommendation system/description.csv")
medications = pd.read_csv("/content/drive/MyDrive/medicine recommendation system/medications.csv")
diets = pd.read_csv("/content/drive/MyDrive/medicine recommendation system/diets.csv")

# Helper functions
def get_closest_symptom(user_input, symptoms_dict):
    matches = get_close_matches(user_input, symptoms_dict.keys(), n=3, cutoff=0.3)
    return matches[0] if matches else None

def helper(dis):
    desc = description[description['Disease'] == dis]['Description']
    desc = " ".join([w for w in desc])

    pre = precautions[precautions['Disease'] == dis][['Precaution_1', 'Precaution_2', 'Precaution_3', 'Precaution_4']]
    pre = [col for col in pre.values]

    med = medications[medications['Disease'] == dis]['Medication']
    med = [med for med in med.values]

    die = diets[diets['Disease'] == dis]['Diet']
    die = [die for die in die.values]

    wrkout = workout[workout['disease'] == dis]['workout']

    return desc, pre, med, die, wrkout

# Symptoms and diseases mapping
symptoms_dict = {'itching': 0, 'skin_rash': 1, 'nodal_skin_eruptions': 2, 'continuous_sneezing': 3,
                'shivering': 4, 'chills': 5, 'joint_pain': 6, 'stomach_pain': 7, 'acidity': 8,
                'ulcers_on_tongue': 9, 'muscle_wasting': 10, 'vomiting': 11, 'burning_micturition': 12,
                'spotting_ urination': 13, 'fatigue': 14, 'weight_gain': 15, 'anxiety': 16,
                'cold_hands_and_feets': 17, 'mood_swings': 18, 'weight_loss': 19, 'restlessness': 20,
                'lethargy': 21, 'patches_in_throat': 22, 'irregular_sugar_level': 23, 'cough': 24,
                'high_fever': 25, 'sunken_eyes': 26, 'breathlessness': 27, 'sweating': 28,
                'dehydration': 29, 'indigestion': 30, 'headache': 31, 'yellowish_skin': 32,
                'dark_urine': 33, 'nausea': 34, 'loss_of_appetite': 35, 'pain_behind_the_eyes': 36,
                'back_pain': 37, 'constipation': 38, 'abdominal_pain': 39, 'diarrhoea': 40,
                'mild_fever': 41, 'yellow_urine': 42, 'yellowing_of_eyes': 43, 'acute_liver_failure': 44,
                'fluid_overload': 45, 'swelling_of_stomach': 46, 'swelled_lymph_nodes': 47, 'malaise': 48,
                'blurred_and_distorted_vision': 49, 'phlegm': 50, 'throat_irritation': 51, 'redness_of_eyes': 52,
                'sinus_pressure': 53, 'runny_nose': 54, 'congestion': 55, 'chest_pain': 56,
                'weakness_in_limbs': 57, 'fast_heart_rate': 58, 'pain_during_bowel_movements': 59,
                'pain_in_anal_region': 60, 'bloody_stool': 61, 'irritation_in_anus': 62, 'neck_pain': 63,
                'dizziness': 64, 'cramps': 65, 'bruising': 66, 'obesity': 67, 'swollen_legs': 68,
                'swollen_blood_vessels': 69, 'puffy_face_and_eyes': 70, 'enlarged_thyroid': 71,
                'brittle_nails': 72, 'swollen_extremeties': 73, 'excessive_hunger': 74,
                'extra_marital_contacts': 75, 'drying_and_tingling_lips': 76, 'slurred_speech': 77,
                'knee_pain': 78, 'hip_joint_pain': 79, 'muscle_weakness': 80, 'stiff_neck': 81,
                'swelling_joints': 82, 'movement_stiffness': 83, 'spinning_movements': 84,
                'loss_of_balance': 85, 'unsteadiness': 86, 'weakness_of_one_body_side': 87,
                'loss_of_smell': 88, 'bladder_discomfort': 89, 'foul_smell_of urine': 90,
                'continuous_feel_of_urine': 91, 'passage_of_gases': 92, 'internal_itching': 93,
                'toxic_look_(typhos)': 94, 'depression': 95, 'irritability': 96, 'muscle_pain': 97,
                'altered_sensorium': 98, 'red_spots_over_body': 99, 'belly_pain': 100,
                'abnormal_menstruation': 101, 'dischromic _patches': 102, 'watering_from_eyes': 103,
                'increased_appetite': 104, 'polyuria': 105, 'family_history': 106, 'mucoid_sputum': 107,
                'rusty_sputum': 108, 'lack_of_concentration': 109, 'visual_disturbances': 110,
                'receiving_blood_transfusion': 111, 'receiving_unsterile_injections': 112, 'coma': 113,
                'stomach_bleeding': 114, 'distention_of_abdomen': 115, 'history_of_alcohol_consumption': 116,
                'fluid_overload.1': 117, 'blood_in_sputum': 118, 'prominent_veins_on_calf': 119,
                'palpitations': 120, 'painful_walking': 121, 'pus_filled_pimples': 122, 'blackheads': 123,
                'scurring': 124, 'skin_peeling': 125, 'silver_like_dusting': 126, 'small_dents_in_nails': 127,
                'inflammatory_nails': 128, 'blister': 129, 'red_sore_around_nose': 130, 'yellow_crust_ooze': 131}

diseases_list = {15: 'Fungal infection', 4: 'Allergy', 16: 'GERD', 9: 'Chronic cholestasis',
                14: 'Drug Reaction', 33: 'Peptic ulcer diseae', 1: 'AIDS', 12: 'Diabetes ',
                17: 'Gastroenteritis', 6: 'Bronchial Asthma', 23: 'Hypertension ', 30: 'Migraine',
                7: 'Cervical spondylosis', 32: 'Paralysis (brain hemorrhage)', 28: 'Jaundice',
                29: 'Malaria', 8: 'Chicken pox', 11: 'Dengue', 37: 'Typhoid', 40: 'hepatitis A',
                19: 'Hepatitis B', 20: 'Hepatitis C', 21: 'Hepatitis D', 22: 'Hepatitis E',
                3: 'Alcoholic hepatitis', 36: 'Tuberculosis', 10: 'Common Cold', 34: 'Pneumonia',
                13: 'Dimorphic hemmorhoids(piles)', 18: 'Heart attack', 39: 'Varicose veins',
                26: 'Hypothyroidism', 24: 'Hyperthyroidism', 25: 'Hypoglycemia', 31: 'Osteoarthristis',
                5: 'Arthritis', 0: '(vertigo) Paroymsal  Positional Vertigo', 2: 'Acne',
                38: 'Urinary tract infection', 35: 'Psoriasis', 27: 'Impetigo'}

# Model Prediction function with error handling
def get_predicted_value(patient_symptoms):
    input_vector = np.zeros(len(symptoms_dict))
    valid_symptoms = []

    for item in patient_symptoms:
        if item in symptoms_dict:
            valid_symptoms.append(item)
            input_vector[symptoms_dict[item]] = 1
        else:
            closest = get_closest_symptom(item, symptoms_dict)
            if closest:
                print(f"Note: Using '{closest}' instead of '{item}'")
                valid_symptoms.append(closest)
                input_vector[symptoms_dict[closest]] = 1
            else:
                print(f"Warning: '{item}' not recognized and no close match found. Skipping.")

    if not valid_symptoms:
        print("Error: No valid symptoms provided for prediction.")
        return None

    try:
        prediction = svc.predict([input_vector])[0]
        return diseases_list.get(prediction, "Unknown Disease")
    except Exception as e:
        print(f"Prediction error: {str(e)}")
        return None

# Main prediction interface
def predict_disease_from_symptoms():
    print("\nAvailable symptoms examples:")
    print(", ".join(list(symptoms_dict.keys())[:10]) + ", ...")

    symptoms = input("\nEnter your symptoms (comma separated): ").strip()
    if not symptoms:
        print("Error: No symptoms entered.")
        return

    user_symptoms = [s.strip().lower() for s in symptoms.split(',')]
    user_symptoms = [symptom.strip("[]'\" ") for symptom in user_symptoms]

    predicted_disease = get_predicted_value(user_symptoms)

    if not predicted_disease:
        print("\nUnable to make a prediction. Please check your symptoms and try again.")
        return

    print("\nProcessing your symptoms...")
    desc, pre, med, die, wrkout = helper(predicted_disease)

    print("\n================= PREDICTED DISEASE =============")
    print(predicted_disease)

    print("\n================= DESCRIPTION ==================")
    print(desc if desc else "No description available")

    print("\n================= PRECAUTIONS ==================")
    if pre and len(pre) > 0 and len(pre[0]) > 0:
        for i, p in enumerate(pre[0], 1):
            print(f"{i}: {p}")
    else:
        print("No precaution information available")

    print("\n================= MEDICATIONS ==================")
    if med and len(med) > 0:
        for i, m in enumerate(med, len(pre[0]) if pre else enumerate(med, 1)):
            print(f"{i}: {m}")
    else:
        print("No medication information available")

    print("\n================= RECOMMENDED DIET =============")
    if die and len(die) > 0:
        start_idx = (len(pre[0]) if pre else 0) + (len(med) if med else 0)
        for i, d in enumerate(die, start_idx + 1):
            print(f"{i}: {d}")
    else:
        print("No diet information available")

    print("\n================= LIFESTYLE RECOMMENDATIONS ====")
    if not wrkout.empty:
        start_idx = (len(pre[0]) if pre else 0) + (len(med) if med else 0) + (len(die) if die else 0)
        for i, w in enumerate(wrkout, start_idx + 1):
            print(f"{i}: {w}")
    else:
        print("No lifestyle recommendations available")

# Run the prediction interface
predict_disease_from_symptoms()

SVC Accuracy: 1.0
RandomForest Accuracy: 1.0
GradientBoosting Accuracy: 1.0
KNeighbors Accuracy: 1.0
MultinomialNB Accuracy: 1.0

Available symptoms examples:
itching, skin_rash, nodal_skin_eruptions, continuous_sneezing, shivering, chills, joint_pain, stomach_pain, acidity, ulcers_on_tongue, ...

Enter your symptoms (comma separated): itching

Processing your symptoms...

================= PREDICTED DISEASE =============
Fungal infection

================= DESCRIPTION ==================
Fungal infection is a common skin condition caused by fungi.

================= PRECAUTIONS ==================
1: bath twice
2: use detol or neem in bathing water
3: keep infected area dry
4: use clean cloths

================= MEDICATIONS ==================
4: ['Antifungal Cream', 'Fluconazole', 'Terbinafine', 'Clotrimazole', 'Ketoconazole']

================= RECOMMENDED DIET =============
6: ['Antifungal Diet', 'Probiotics', 'Garlic', 'Coconut oil', 'Turmeric']

================= LIFESTYLE RECOMMEND

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


In [2]:
predict_disease_from_symptoms()


Available symptoms examples:
itching, skin_rash, nodal_skin_eruptions, continuous_sneezing, shivering, chills, joint_pain, stomach_pain, acidity, ulcers_on_tongue, ...

Enter your symptoms (comma separated): nodal skin ertyion
Note: Using 'nodal_skin_eruptions' instead of 'nodal skin ertyion'

Processing your symptoms...

================= PREDICTED DISEASE =============
Fungal infection

================= DESCRIPTION ==================
Fungal infection is a common skin condition caused by fungi.

================= PRECAUTIONS ==================
1: bath twice
2: use detol or neem in bathing water
3: keep infected area dry
4: use clean cloths

================= MEDICATIONS ==================
4: ['Antifungal Cream', 'Fluconazole', 'Terbinafine', 'Clotrimazole', 'Ketoconazole']

================= RECOMMENDED DIET =============
6: ['Antifungal Diet', 'Probiotics', 'Garlic', 'Coconut oil', 'Turmeric']

================= LIFESTYLE RECOMMENDATIONS ====
7: Avoid sugary foods
8: Consume probio

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(
